In [2]:
library(knitr)
library(sqldf)
library(ggplot2)
library(mongolite)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(tidyr)
library(jsonlite)
library(lubridate)
library(gridExtra)
library(GGally)
library(scales)
library(memisc)
 library(sp)
library(rgeos)
library(rgdal)
library(maptools)
library(RColorBrewer)
library(maps)
library(grid)
library(cluster)
library(StatMatch)
library(fastcluster)
library(factoextra)
library(stringr)
library(googleVis)
library(RMySQL)
library(data.table)
library(dbplyr)
library(DBI)
library(httr)
library(ldatuning)
library(udpipe)
library(egg)
library(tm)
library(tidytext)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite

Attaching package: 'dplyr'

The following object is masked from 'package:gridExtra':

    combine

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'GGally'

The following object is masked from 'package:dplyr':

    nasa

Loading required package: lattice
Loading required package: MASS

Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select


Attaching package: 'memisc'

The following object is masked from 'package:scales':

    percent

The following object is masked from 'package:lubridate':

    is.interval

The following objects are masked from 'package:dplyr':

    collect, recode, rename, syms

The following objec

In [ ]:
#### Some examples of cleaning the twitter text

In [3]:
ferg_cols_loc <- fread(file="../Preprocessing/Files/Source/ferg_cols_location.csv", sep =",", header = TRUE, 
              stringsAsFactors=FALSE,
             showProgress=getOption("datatable.showProgress", interactive()), data.table=FALSE)

write.csv(ferg_cols_loc, "C:\\MathLabASUS\\Courses\\Ryerson\\Project\\Data\\Files\\Source\\ferg_cols_location.csv")

In [7]:
str(ferg_cols_loc)

'data.frame':	3884008 obs. of  32 variables:
 $ rowno                      : num  1 2 3 4 5 6 7 8 9 10 ...
 $ user.id                    :integer64 1539100118 9439062 63380850 1539100118 9439062 26312359 40652267 296368494 ... 
 $ user.location              : chr  "Columbus, OH" "Atlanta, GA" "CaLove" "Columbus, OH" ...
 $ user.description           : chr  "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." "Husband. Father. Prelate. Human. Company builder, 2 exits. Author @kingonomics. Inclusive ecosystem builder @th"| __truncated__ "Jealous Guardian of our Democracy #RIPTrayvonMartin #ImAlwaysWithHer" "Advocate for survivors of sexual violence. Breaker of silence, destroyer of shame." ...
 $ created_at                 : chr  "Sat Aug 09 22:33:06 +0000 2014" "Sat Aug 09 22:59:13 +0000 2014" "Sat Aug 09 22:00:02 +0000 2014" "Sat Aug 09 22:28:42 +0000 2014" ...
 $ id                         : num  4.98e+17 4.98e+17 4.98e+17 4.98e+17 4.98e+17 ...
 $ user.

In [6]:
ferg_cols_loc$V1 <- NULL
ferg_cols_loc$V1 <- NULL

In [4]:
full_text <- data.frame(full_text = ferg_cols_loc$full_text)

In [5]:
ferg_cols_loc$textnum <- seq(1, nrow(ferg_cols_loc), 1) 

In [6]:
head(ferg_cols_loc$textnum)

[1] 1 2 3 4 5 6

In [7]:
###### Sample number of locationa

full_text,textnum_unq
Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,1
"@PhaedraParks While there, please tune in today's killing of the young black male in #Ferguson. #SaveOurBoys",2
RT @PhenomMusik: @NAACP pictures speak 1000 words! Unarmed teen shot 10 times by #Ferguson police http://t.co/COLJqL4N3h,3
SICK and TIRED of police killing unarmed people. #ferguson #walmart,4
"Well, here we go again. #Ferguson",5
Damn shame... This coulda been my son or yours. Our condolences to the family... #ferguson #policeâ€¦ http://t.co/jpr9daiypl,6


In [80]:
str(ferg_cols_samp)

'data.frame':	1553147 obs. of  38 variables:
 $ V1                         : int  1191549 3795103 1240177 3778994 1534743 3308538 2567906 3237089 2400755 3841048 ...
 $ rowno                      : num  1191549 1217960 1240177 675679 1534743 ...
 $ user.id                    :integer64 277674677 67976889 18388012 528894289 33712376 33712376 33712376 1287620058 ... 
 $ user.location              : chr  "Abbeville, SC" "Aberdeen, Maryland" "Aberdeen, MD" "Aberdeen, North Carolina" ...
 $ user.description           : chr  "Servant, Husband, Father, Son, Pastor, Founder. Living Everyday Achieving Dreams (L.E.A.D.) - ing by example!" "I still don't wish death on them. I just reflect on them." "I walked in universes where the laws of physics were devised by the mind of a mad man" "Fire Fighter. Dorado Slayer. Pacifico Drinker. Zombie Ventilator." ...
 $ created_at                 : chr  "Thu Aug 14 14:48:31 +0000 2014" "Thu Aug 14 15:26:22 +0000 2014" "Thu Aug 14 15:57:51 +0000 2014" "Thu Au

In [7]:
ferg_cols_loc$hashtags <- str_extract_all(ferg_cols_loc$full_text, "#[[:graph:]]*")

In [8]:
ferg_cols_loc$hashtags <- gsub("[#|.|,]", "", ferg_cols_loc$hashtags)

In [9]:
ferg_cols_loc$hashtags <- gsub("\\s{1,}", " ", ferg_cols_loc$hashtags)

In [10]:
ferg_cols_loc$keyword_pos_hash <- str_locate_all(ferg_cols_loc$full_text, "#[[:graph:]]*")

In [11]:
ferg_cols_loc$keyword_pos_hash <- as.character(ferg_cols_loc$keyword_pos_hash)

In [12]:
ferg_cols_loc$keyword_count_hash <- str_count(ferg_cols_loc$full_text, "#[[:graph:]]*")

In [23]:
ferg_cols_loc$full_text2 <- iconv(ferg_cols_loc$full_text, to = "ASCII//TRANSLIT")

In [24]:
ferg_cols_loc$full_text2 <- gsub("[\\||?|\"]", " ", ferg_cols_loc$full_text2)

In [25]:
ferg_cols_loc$full_text2 <- gsub("@[a-zA-Z0-9_]{1,15}", " ", ferg_cols_loc$full_text2)

In [26]:
ferg_cols_loc$full_text2 <- gsub("(http|https)://[[:graph:]]*", " ", ferg_cols_loc$full_text2)  

In [27]:
ferg_cols_loc$full_text2 <- gsub("\\s{1,}", " ", ferg_cols_loc$full_text2)

In [29]:
ferg_cols_loc$full_text_nohash <- gsub("#[a-zA-Z0-9_]*", " ", ferg_cols_loc$full_text2)

In [30]:
ferg_cols_loc$full_text2 <- gsub("[\\.,:;\\?!\\\\\\|\\/`=\\*\\+-^_~\\\"\\\'\\[\\]\\{\\}\\(\\)<>@$]*", "", ferg_cols_loc$full_text2)

In [209]:
head(ferg_cols_loc$full_text2)

[1] "Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson"
[2] " While there, please tune in today's killing of the young black male in #Ferguson. #SaveOurBoys"                                 
[3] "RT : pictures speak 1000 words! Unarmed teen shot 10 times by #Ferguson police "                                                 
[4] "SICK and TIRED of police killing unarmed people. #ferguson #walmart"                                                             
[5] "Well, here we go again. #Ferguson"                                                                                               
[6] "Damn shame... This coulda been my son or yours. Our condolences to the family... #ferguson #policea "

In [31]:
ferg_cols_loc$full_text2 <- gsub(":", "", ferg_cols_loc$full_text2)

In [32]:
ferg_cols_loc$full_text2 <- gsub("\\s{1,}", " ", ferg_cols_loc$full_text2)

In [33]:
ferg_cols_loc$tweet_type <- ifelse(ferg_cols_loc$full_text %like% "RT", "RT", 
                                    ifelse(ferg_cols_loc$full_text %like% "@", "SB", "All"))

In [213]:
table(ferg_cols_loc$tweet_type)


    All      RT      SB 
 561098 3045097  277813 

In [34]:
table(ferg_cols_loc$tweet_type)


    All      RT      SB 
 561098 3045097  277813 

In [42]:
head(ferg_cols_loc[ ,c("full_text_nohash", "textnum")])

full_text_nohash,textnum
Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children.,1
"While there, please tune in today's killing of the young black male in .",2
RT : pictures speak 1000 words! Unarmed teen shot 10 times by police,3
SICK and TIRED of police killing unarmed people.,4
"Well, here we go again.",5
Damn shame... This coulda been my son or yours. Our condolences to the family...,6


###### Remove retweet - RT

In [36]:
ferg_cols_loc$full_text2 <- gsub("RT", "", ferg_cols_loc$full_text2)

###### Trim whitespaces

In [37]:
ferg_cols_loc$full_text2 <- trimws(ferg_cols_loc$full_text2, which = "both")

In [38]:
ferg_cols_loc$full_text2 <- iconv(ferg_cols_loc$full_text2, "latin1", "ASCII", sub="")

In [39]:
ferg_cols_loc$full_text2 <- gsub("\\s{1,}", " ", ferg_cols_loc$full_text2)

In [188]:
full_text <- data.frame(text = ferg_cols_loc$full_text2)